In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano.tensor as tt
import warnings

from scipy.stats import beta
from scipy.special import expit
from matplotlib import gridspec
from IPython.display import Image

%matplotlib inline
plt.style.use('seaborn-white')

color = '#87ceeb'

f_dict = {'size':16}

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\gianc\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\gianc\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}

In [2]:
def plot_mustache(o, k, j, width=.65, ax=None):
    for i in np.arange(0, len(o), int(len(o)*.1)):
        a = o[i]*(k[i]-2)+1
        b = (1-o[i])*(k[i]-2)+1
        rv = beta(a,b)
                
        yrange = np.linspace(rv.ppf(0.025), rv.ppf(0.975), 100)
        xrange = rv.pdf(yrange)
        
        # When the SD of a group is large compared to others, then the top of its mustache is relatively
        # low and does not plot well together with low SD groups.
        # Scale the xrange so that the 'height' of the all mustaches is 0.75
        xrange_scaled = xrange*(width/xrange.max())
        
        # Using the negative value to flip the mustache in the right direction.
        ax.plot(-xrange_scaled+j, yrange, color=color, alpha=.5)

# Import Burst 1 data

In [3]:
df = pd.read_csv('C:/Users/gianc/Box Sync/GP_desktop/SAA 2021/Calculated compliance dataset/b1compliance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 3 columns):
id                  265 non-null int64
b1_run_ins          265 non-null int64
b1_survey_counts    265 non-null int64
dtypes: int64(3)
memory usage: 6.3 KB


In [5]:
df.head(5)

,id,b1_run_ins,b1_survey_counts
0,3001,0,67
1,3002,43,57
2,3003,47,70
3,3004,52,74
4,3005,44,91


In [8]:
# convert columns "b1_run_ins" and "b1_survey_counts" to float64 dtype
df = df.astype({"b1_run_ins": float, "b1_survey_counts": float})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 3 columns):
id                  265 non-null int64
b1_run_ins          265 non-null float64
b1_survey_counts    265 non-null float64
dtypes: float64(2), int64(1)
memory usage: 6.3 KB


In [10]:
X = df['b1_run_ins']
y = df['b1_survey_counts']

meanx = X.mean()
scalex = X.std()
zX = ((X-meanx)/scalex)

In [15]:
# check if this worked
zX

0     -2.000831
1      0.977934
2      1.255029
3      1.601397
4      1.047208
5      0.770113
6      1.047208
7     -1.031000
8      0.285198
9     -1.308095
10    -0.338264
11     0.908661
12     0.215925
13     0.423745
14    -1.585189
15    -0.199717
16    -0.407538
17    -0.684632
18     0.770113
19    -1.031000
20     0.839387
21     0.908661
22    -1.100274
23     0.354472
24    -0.546085
25     0.008104
26    -0.407538
27    -1.031000
28     1.255029
29    -1.100274
         ...   
235    1.047208
236    0.908661
237    1.532123
238   -0.961727
239   -1.100274
240    0.354472
241    0.008104
242    0.908661
243   -0.684632
244    1.324302
245   -0.961727
246   -0.546085
247    0.977934
248   -1.031000
249   -1.031000
250   -0.061170
251    1.047208
252   -0.823180
253    1.047208
254    0.700840
255    0.770113
256    1.739944
257   -1.100274
258   -1.031000
259   -0.961727
260   -1.031000
261   -0.961727
262    1.947765
263    1.462850
264   -0.753906
Name: b1_run_ins, Length

In [20]:
with pm.Model() as model_weight:
    
    zbeta0 = pm.Normal('zbeta0', mu=0, sd=2)
    zbetaj = pm.Normal('zbetaj', mu=0, sd=2)
        
    p = pm.invlogit(zbeta0 + zbetaj*zX[:,1])
        
    likelihood = pm.Bernoulli('likelihood', p, observed=y.values)

ValueError: Can only tuple-index with a MultiIndex

In [ ]:
with model_weight:
    trace1 = pm.sample(3000, cores=4)

In [ ]:
# Transform parameters back to original scale
beta0 = trace1['zbeta0'] - trace1['zbetaj']*meanx[1]/scalex[1]
betaj = (trace1['zbetaj']/scalex[1])

plt.figure(figsize=(10,10))
# Define gridspec
gs = gridspec.GridSpec(3, 4)
ax1 = plt.subplot(gs[:2,:4])
ax2 = plt.subplot(gs[2,:2])
ax3 = plt.subplot(gs[2,2:])

ax1.scatter(df.weight, df.male, s=100, edgecolor='k', facecolors='None', lw=1)

# Take 20 values from the posterior distribution and plot the lines
n_curves = 20
tr_len = len(trace1)
stepIdxVec = np.arange(0, tr_len, tr_len//n_curves)
weight_span = np.arange(df.weight.min(), df.weight.max())
weights = np.tile(weight_span.reshape(-1,1), (1,n_curves))
# The expit function from scipy.special calculates the inverse of the logit function
p = expit(beta0[stepIdxVec] + betaj[stepIdxVec]*weights)
ax1.plot(weights, p, c=color)

ax1.axhline(y=0.5, color='k', linestyle='dotted')

# Look up weights for which of the posterior probabilities is (close to) 0.5
decision_boundary = weight_span[np.any(np.isclose(p, 0.5, atol=0.01), axis=1)]
# Highlight the weightspan
ax1.axvspan(decision_boundary.min(), decision_boundary.max(),
            0, 0.5, color=color, alpha=0.3)

ax1.set_xlabel('Weight')
ax1.set_ylabel('Male')
ax1.set_title('Data with Post. Pred.')

pm.plot_posterior(beta0, point_estimate='mode', ax=ax2, color=color)
ax2.set_title('Intercept', fontdict=f_dict)
ax2.set_xlabel(r'$\beta_0$', fontdict=f_dict)

pm.plot_posterior(betaj, point_estimate='mode', ax=ax3, color=color)
ax3.set_title('Weight', fontdict=f_dict)
ax3.set_xlabel(r'$\beta_1$', fontdict=f_dict);

plt.tight_layout();